# Prediction Models (CIDEIM)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV

In [2]:
# Import utilities
sys.path.insert(0, '../')
from helpers import preprocessing
from helpers import sets
from helpers import metrics

## Preprocessing

### Import dataset

In [3]:
df = pd.read_csv("../../data_sets/ds3.csv")

### Encode target values

In [4]:
preprocessing.encoding(df)

### Test-Train split

In [5]:
from sklearn.model_selection import train_test_split

subset = sets.cideim

# Input data
X = df[subset]

# Target variable
y = df['y']

# Split dataset in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Scale

In [6]:
pareto = preprocessing.ParetoScaler()
X_train = pareto.fit_transform(X_train)
X_test = pareto.transform(X_test)

## Decision Tree

In [7]:
from sklearn.tree import DecisionTreeClassifier

In [8]:
dt_model = DecisionTreeClassifier(criterion="gini", splitter="best")
dt_model.fit(X_train, y_train)

DecisionTreeClassifier()

In [9]:
y_pred = dt_model.predict(X_test)
metrics.metrics_report(dt_model, X, y, y_test, y_pred)

=== ACCURACY
Mean:		0.6
STD:		0.10690449676496977
=== PRECISION
Mean:		0.7866666666666666
STD:		0.1758787461103056
=== RECALL
Mean:		0.74
STD:		0.21540659228538014
=== F1
Mean:		0.7354545454545454
STD:		0.07342843820475882


## Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression

In [11]:
lg_model = LogisticRegression(penalty="l1", solver="liblinear")
lg_model.fit(X_train, y_train)

LogisticRegression(penalty='l1', solver='liblinear')

In [12]:
y_pred = lg_model.predict(X_test)
metrics.metrics_report(lg_model, X, y, y_test, y_pred)

=== ACCURACY
Mean:		0.8285714285714286
STD:		0.10690449676496974
=== PRECISION
Mean:		0.8699999999999999
STD:		0.10770329614269006
=== RECALL
Mean:		0.82
STD:		0.22271057451320087
=== F1
Mean:		0.8222222222222222
STD:		0.1333333333333334


## Naive Bayes

In [13]:
from sklearn.naive_bayes import GaussianNB

In [14]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

GaussianNB()

In [15]:
y_pred = nb_model.predict(X_test)
metrics.metrics_report(nb_model, X, y, y_test, y_pred)

=== ACCURACY
Mean:		0.9142857142857143
STD:		0.11428571428571428
=== PRECISION
Mean:		0.96
STD:		0.07999999999999999
=== RECALL
Mean:		0.9099999999999999
STD:		0.11135528725660043
=== F1
Mean:		0.9314285714285715
STD:		0.08590455073355943


## Support Vector Classifier (SVC)

In [16]:
from sklearn.svm import SVC

In [17]:
svc_model = SVC(kernel="poly", degree=2)
svc_model.fit(X_train, y_train)

SVC(degree=2, kernel='poly')

In [18]:
y_pred = svc_model.predict(X_test)
metrics.metrics_report(svc_model, X, y, y_test, y_pred)

=== ACCURACY
Mean:		0.8285714285714285
STD:		0.10690449676496974
=== PRECISION
Mean:		0.9099999999999999
STD:		0.11135528725660043
=== RECALL
Mean:		0.8300000000000001
STD:		0.15362291495737218
=== F1
Mean:		0.8555555555555555
STD:		0.09525793444156806


## Artificial Neural Network (ANN)

In [19]:
import tensorflow as tf

2022-08-13 14:55:18.422288: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-13 14:55:18.422313: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [20]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer

In [21]:
def build_model(n_hidden=1, n_neurons=6, learning_rate=3e-3, input_shape=X_train.shape[1]):
  model = Sequential()
  model.add(InputLayer(input_shape=input_shape))
  for layer in range(n_hidden):
    model.add(Dense(n_neurons, activation="relu"))
  model.add(Dense(1, activation='sigmoid'))
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
  return model

In [22]:
from keras.wrappers.scikit_learn import KerasClassifier
ann_model = KerasClassifier(build_model)

/tmp/ipykernel_3994/2775760517.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ann_model = KerasClassifier(build_model)


In [23]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
from keras.callbacks import EarlyStopping

param_distribs = {
  "n_hidden": [0, 1, 2, 3],
  "n_neurons": np.arange(1, 10),
  "learning_rate": reciprocal(3e-4, 3e-2),
}
rnd_search_cv = RandomizedSearchCV(ann_model, param_distribs, n_iter=10,
cv=10)
rnd_search_cv.fit( X_train, y_train, batch_size = 32, epochs=100,                  
                  validation_split=0.05,  # using only for earlystopping
                  callbacks=[EarlyStopping(patience=5)])

2022-08-13 14:55:21.082732: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-13 14:55:21.082812: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-13 14:55:21.082874: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tron): /proc/driver/nvidia/version does not exist
2022-08-13 14:55:21.083631: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 67.4571 - accuracy: 0.5217 - val_loss: 1.0261e-24 - val_accuracy: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 34ms/step - loss: 50.6434 - accuracy: 0.4783 - val_loss: 1.3206e-13 - val_accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 37ms/step - loss: 40.3289 - accuracy: 0.5652 - val_loss: 2.6453e-10 - val_accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 38ms/step - loss: 30.2836 - accuracy: 0.5217 - val_loss: 2.1009e-11 - val_accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 37ms/step - loss: 21.6315 - accuracy: 0.5217 - val_loss: 3.8835e-13 - val_accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 70ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 1/100
1/1 [==============================] - 1s 809ms/step - loss: 98.8869 - accuracy: 0.6957 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/100
1/1 [================

1/1 [==============================] - 0s 22ms/step - loss: 2.9047e-08 - accuracy: 1.0000
Epoch 1/100
1/1 [==============================] - 1s 707ms/step - loss: 15.9458 - accuracy: 0.6087 - val_loss: 7.3229e-04 - val_accuracy: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 7.4497 - accuracy: 0.6957 - val_loss: 0.0019 - val_accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 6.4042 - accuracy: 0.7391 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 6.3964 - accuracy: 0.6957 - val_loss: 9.0497e-04 - val_accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 31ms/step - loss: 5.8249 - accuracy: 0.6522 - val_loss: 2.4184e-04 - val_accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 26ms/step - loss: 4.9308 - accuracy: 0.6087 - val_loss: 1.7440e-05 - val_accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0

1/1 [==============================] - 0s 28ms/step - loss: 11.2421 - accuracy: 0.6667 - val_loss: 1.1872e-05 - val_accuracy: 1.0000
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - loss: 9.2104 - accuracy: 0.6667 - val_loss: 0.0310 - val_accuracy: 1.0000
Epoch 12/100
1/1 [==============================] - 0s 27ms/step - loss: 7.1662 - accuracy: 0.6667 - val_loss: 3.2497 - val_accuracy: 0.5000
Epoch 13/100
1/1 [==============================] - 0s 23ms/step - loss: 7.4076e-22 - accuracy: 1.0000
Epoch 1/100
1/1 [==============================] - 1s 622ms/step - loss: 206.4039 - accuracy: 0.3913 - val_loss: 218.0606 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 190.9538 - accuracy: 0.3913 - val_loss: 196.6718 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 26ms/step - loss: 175.6612 - accuracy: 0.3478 - val_loss: 175.2937 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [====================

1/1 [==============================] - 0s 32ms/step - loss: 0.6598 - accuracy: 0.6957 - val_loss: 0.5976 - val_accuracy: 1.0000
Epoch 30/100
1/1 [==============================] - 0s 28ms/step - loss: 0.6588 - accuracy: 0.6957 - val_loss: 0.5946 - val_accuracy: 1.0000
Epoch 31/100
1/1 [==============================] - 0s 29ms/step - loss: 0.6579 - accuracy: 0.6957 - val_loss: 0.5917 - val_accuracy: 1.0000
Epoch 32/100
1/1 [==============================] - 0s 29ms/step - loss: 0.6569 - accuracy: 0.6957 - val_loss: 0.5888 - val_accuracy: 1.0000
Epoch 33/100
1/1 [==============================] - 0s 30ms/step - loss: 0.6560 - accuracy: 0.6957 - val_loss: 0.5859 - val_accuracy: 1.0000
Epoch 34/100
1/1 [==============================] - 0s 30ms/step - loss: 0.6551 - accuracy: 0.6957 - val_loss: 0.5831 - val_accuracy: 1.0000
Epoch 35/100
1/1 [==============================] - 0s 27ms/step - loss: 0.6542 - accuracy: 0.6957 - val_loss: 0.5802 - val_accuracy: 1.0000
Epoch 36/100
1/1 [========

...

1/1 [==============================] - 0s 33ms/step - loss: 68.7078 - accuracy: 0.3913 - val_loss: 90.9289 - val_accuracy: 0.0000e+00
Epoch 63/100
1/1 [==============================] - 0s 34ms/step - loss: 65.5040 - accuracy: 0.3913 - val_loss: 87.3656 - val_accuracy: 0.0000e+00
Epoch 64/100
1/1 [==============================] - 0s 33ms/step - loss: 62.5523 - accuracy: 0.3913 - val_loss: 83.8737 - val_accuracy: 0.0000e+00
Epoch 65/100
1/1 [==============================] - 0s 35ms/step - loss: 59.6653 - accuracy: 0.3913 - val_loss: 80.4485 - val_accuracy: 0.0000e+00
Epoch 66/100
1/1 [==============================] - 0s 35ms/step - loss: 56.8291 - accuracy: 0.3913 - val_loss: 77.0854 - val_accuracy: 0.0000e+00
Epoch 67/100
1/1 [==============================] - 0s 33ms/step - loss: 54.0403 - accuracy: 0.3913 - val_loss: 73.7805 - val_accuracy: 0.0000e+00
Epoch 68/100
1/1 [==============================] - 0s 32ms/step - loss: 51.3320 - accuracy: 0.3913 - val_loss: 70.5472 - val_accur

1/1 [==============================] - 0s 31ms/step - loss: 3.1932 - accuracy: 0.5000 - val_loss: 0.6914 - val_accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 39ms/step - loss: 2.1818 - accuracy: 0.5417 - val_loss: 0.6906 - val_accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.5760 - accuracy: 0.5417 - val_loss: 0.6897 - val_accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1148 - accuracy: 0.5833 - val_loss: 0.6889 - val_accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 0.7741 - accuracy: 0.5833 - val_loss: 0.6881 - val_accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 0.6919 - accuracy: 0.6250 - val_loss: 0.6872 - val_accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 0.6917 - accuracy: 0.6250 - val_loss: 0.6864 - val_accuracy: 1.0000
Epoch 9/100
1/1 [===============

1/1 [==============================] - 0s 37ms/step - loss: 0.6826 - accuracy: 0.6250 - val_loss: 0.6468 - val_accuracy: 1.0000
Epoch 61/100
1/1 [==============================] - 0s 39ms/step - loss: 0.6824 - accuracy: 0.6250 - val_loss: 0.6461 - val_accuracy: 1.0000
Epoch 62/100
1/1 [==============================] - 0s 34ms/step - loss: 0.6822 - accuracy: 0.6250 - val_loss: 0.6453 - val_accuracy: 1.0000
Epoch 63/100
1/1 [==============================] - 0s 34ms/step - loss: 0.6821 - accuracy: 0.6250 - val_loss: 0.6446 - val_accuracy: 1.0000
Epoch 64/100
1/1 [==============================] - 0s 39ms/step - loss: 0.6819 - accuracy: 0.6250 - val_loss: 0.6439 - val_accuracy: 1.0000
Epoch 65/100
1/1 [==============================] - 0s 36ms/step - loss: 0.6818 - accuracy: 0.6250 - val_loss: 0.6432 - val_accuracy: 1.0000
Epoch 66/100
1/1 [==============================] - 0s 36ms/step - loss: 0.6816 - accuracy: 0.6250 - val_loss: 0.6425 - val_accuracy: 1.0000
Epoch 67/100
1/1 [========

Epoch 17/100
1/1 [==============================] - 0s 31ms/step - loss: 23.3276 - accuracy: 0.3333 - val_loss: 5.7045 - val_accuracy: 0.0000e+00
Epoch 18/100
1/1 [==============================] - 0s 29ms/step - loss: 22.0407 - accuracy: 0.3333 - val_loss: 4.8795 - val_accuracy: 0.0000e+00
Epoch 19/100
1/1 [==============================] - 0s 31ms/step - loss: 20.7968 - accuracy: 0.3333 - val_loss: 4.0738 - val_accuracy: 0.0000e+00
Epoch 20/100
1/1 [==============================] - 0s 27ms/step - loss: 19.5818 - accuracy: 0.2917 - val_loss: 3.2866 - val_accuracy: 0.0000e+00
Epoch 21/100
1/1 [==============================] - 0s 33ms/step - loss: 18.4483 - accuracy: 0.3333 - val_loss: 2.5245 - val_accuracy: 0.0000e+00
Epoch 22/100
1/1 [==============================] - 0s 28ms/step - loss: 17.3958 - accuracy: 0.3750 - val_loss: 1.8021 - val_accuracy: 0.0000e+00
Epoch 23/100
1/1 [==============================] - 0s 28ms/step - loss: 16.4679 - accuracy: 0.4583 - val_loss: 1.1682 - val

RandomizedSearchCV(cv=10,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f0b001e2cd0>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f0b001f8760>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([1, 2, 3, 4, 5, 6, 7, 8, 9])})

In [24]:
(rnd_search_cv.best_params_, rnd_search_cv.best_score_)

({'learning_rate': 0.00688259763294182, 'n_hidden': 3, 'n_neurons': 9},
 0.7333333373069764)

In [25]:
ann = rnd_search_cv.best_estimator_.model

In [26]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 10)

Epoch 1/10


2022-08-13 14:59:49.041266: W tensorflow/core/common_runtime/forward_type_inference.cc:231] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_BOOL
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_LEGACY_VARIANT
    }
  }
}

	while inferring type of node 'binary_crossentropy/cond/output/_10'


1/1 [==============================] - 1s 672ms/step - loss: 0.6149 - accuracy: 0.6071
Epoch 2/10
1/1 [==============================] - 0s 6ms/step - loss: 0.5441 - accuracy: 0.6429
Epoch 3/10
1/1 [==============================] - 0s 5ms/step - loss: 0.5245 - accuracy: 0.6429
Epoch 4/10
1/1 [==============================] - 0s 5ms/step - loss: 0.5201 - accuracy: 0.6429
Epoch 5/10
1/1 [==============================] - 0s 6ms/step - loss: 0.5067 - accuracy: 0.6429
Epoch 6/10
1/1 [==============================] - 0s 7ms/step - loss: 0.4970 - accuracy: 0.6429
Epoch 7/10
1/1 [==============================] - 0s 7ms/step - loss: 0.4777 - accuracy: 0.6429
Epoch 8/10
1/1 [==============================] - 0s 6ms/step - loss: 0.4560 - accuracy: 0.6429
Epoch 9/10
1/1 [==============================] - 0s 5ms/step - loss: 0.4314 - accuracy: 0.6429
Epoch 10/10
1/1 [==============================] - 0s 7ms/step - loss: 0.4382 - accuracy: 0.6429


In [27]:
metrics.ann_metrics_report(ann, X, y, 5)

1/1 [==============================] - 0s 28ms/step
=== ACCURACY
MEAN (accuracy):		0.6571428571428571
STD (accuracy):		0.42952275366779735
